# 第11章：ロジスティック回帰

## 11.1 ロジスティック回帰のハイパーパラメータを変更してAUCを向上できないか試してみよう。

### 【解答例】

- 途中までは書籍本編と同様の処理を実行する。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import StandardScaler # 標準化用モジュール
from sklearn.model_selection import train_test_split # ホールドアウト用モジュール
from sklearn.linear_model import LogisticRegression as logr # ロジスティック回帰用モジュール
from sklearn.metrics import accuracy_score, roc_auc_score # 評価指標用モジュール

In [ ]:
# 前処理済み中間データのdictを読み取る
pp_data_dict = joblib.load("../intermediate/pp_data_dict.pkl3")

# dictのvalueに格納されたロジスティック回帰向け中間データを読み取る
lgr_train_df = pp_data_dict["lgr"]["train"]
lgr_test_df = pp_data_dict["lgr"]["test"]

# 6:4の割合でホールドアウト法を行う.
lgr_train_train_df, lgr_train_valid_df = train_test_split(lgr_train_df, test_size=0.4, random_state=57, shuffle=True)

lgr_train_df.shape, lgr_train_train_df.shape, lgr_train_valid_df.shape, lgr_test_df.shape

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"

# 説明変数をfeaturesという変数に格納する
lgr_features = lgr_train_df.columns.tolist()

# customer_idとbuy_flagは説明変数ではない為削除する
lgr_features.remove("customer_id")
lgr_features.remove("buy_flag")

In [ ]:
# ハイパーパラメータチューニング
# グリッドサーチ
# 探索するハイパーパラメータの候補をリストに格納する.
lgr_params_list = [{"penalty": "l2", "C": ii, "random_state": 57} for ii in np.arange(0.5, 1.0, 0.005)]
lgr_params_list

In [ ]:
# グリッドサーチの結果を格納するリスト
gs_result_list = []

# グリッドサーチの実施
for params_dict in lgr_params_list:
    # 定義する
    tmp_lgr_clf = logr(**params_dict, max_iter=1000)
    # 学習する
    tmp_lgr_clf.fit(lgr_train_train_df[lgr_features], lgr_train_train_df[target])
    
    # train-trainを予測する（predict関数）
    train_pred_y = tmp_lgr_clf.predict(lgr_train_train_df[lgr_features])
    
    # train-trainを予測する（predict_proba関数）
    train_proba_y = tmp_lgr_clf.predict_proba(lgr_train_train_df[lgr_features]).T[1]
    
    # train-validを予測する（predict関数）
    valid_pred_y = tmp_lgr_clf.predict(lgr_train_valid_df[lgr_features])
    
    # train-validを予測する（predict_proba関数）
    valid_proba_y = tmp_lgr_clf.predict_proba(lgr_train_valid_df[lgr_features]).T[1]
    
    # train-trainを検証する
    train_accuracy_val = accuracy_score(lgr_train_train_df[target], train_pred_y)
    train_auc_val = roc_auc_score(lgr_train_train_df[target], train_proba_y)
    
    # train-validを検証する
    valid_accuracy_val = accuracy_score(lgr_train_valid_df[target], valid_pred_y)
    valid_auc_val = roc_auc_score(lgr_train_valid_df[target], valid_proba_y)
    
    # リストに格納する
    gs_result_list += [[params_dict, params_dict["C"], train_accuracy_val, train_auc_val, valid_accuracy_val, valid_auc_val]]

In [ ]:
# 結果をDataFrameに格納する
gs_result_df = pd.DataFrame(gs_result_list, columns=["params_dict", "L2_regularization_strength", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"])

# 各パラメータ値ごとのvalidのAUCを確認する
gs_result_df[["params_dict", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"]].sort_values(by="valid_auc", ascending=False).reset_index(drop=True).head(10)

- どうやらCが0.765の時に最もvalid_aucが高くなるということが分かった。

### 【解説】

- 本書本編では0.005～0.5の範囲で、この章末問題では0.5～1.0の範囲で、それぞれ0.005刻みで別々に変化させてみましたが、もちろん一括で0.005～1.0の範囲で0.005刻みで変化させてみても良いでしょう。
- なお、LogisticRegressionを定義する際に、max_iter=1000を設定しました。これを設定しなくても一応結果は出るのですが、ConvergenceWarningというモデルが収束しないエラーが出ます。max_iterはデフォルトが100に設定されていますが、明示的に1000に設定して反復回数を増やすことで対応しました（その分少し時間がかかるようになります）。

## 11.2 第8章の章末問題で自作した特徴量を使ってロジスティック回帰モデルを使って精度を検証してみよう。

### 【解答例】

- 第8章の章末問題で作成した中間データ（pickleファイル）を使用する

In [ ]:
# 前処理済み中間データのdictを読み取る
shomatsu_pp_data_dict = joblib.load("../intermediate/shomatsu_pp_data_dict.pkl3")

# dictのvalueに格納されたロジスティック回帰向け中間データを読み取る
shomatsu_lgr_train_df = shomatsu_pp_data_dict["lgr"]["train"]
shomatsu_lgr_test_df = shomatsu_pp_data_dict["lgr"]["test"]

# 6:4の割合でホールドアウト法を行う.
shomatsu_lgr_train_train_df, shomatsu_lgr_train_valid_df = train_test_split(
    shomatsu_lgr_train_df,
    test_size=0.4,
    random_state=57,
    shuffle=True
)

shomatsu_lgr_train_df.shape, shomatsu_lgr_train_train_df.shape, shomatsu_lgr_train_valid_df.shape, shomatsu_lgr_test_df.shape

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"

# 説明変数をfeaturesという変数に格納する
shomatsu_lgr_features = shomatsu_lgr_train_df.columns.tolist()

# customer_idとbuy_flagは説明変数ではない為削除する
shomatsu_lgr_features.remove("customer_id")
shomatsu_lgr_features.remove("buy_flag")

In [ ]:
# ハイパーパラメータチューニング
# グリッドサーチ
# 探索するハイパーパラメータの候補をリストに格納する.
shomatsu_lgr_params_list = [{"penalty": "l2", "C": ii, "random_state": 57} for ii in np.arange(0.005, 0.5, 0.005)]

# グリッドサーチの結果を格納するリスト．
shomatsu_gs_result_list = []

# グリッドサーチの実施
for params_dict in shomatsu_lgr_params_list:
    # 定義する
    tmp_shomatsu_lgr_clf = logr(**params_dict, max_iter=1000)
    # 学習する
    tmp_shomatsu_lgr_clf.fit(shomatsu_lgr_train_train_df[shomatsu_lgr_features], shomatsu_lgr_train_train_df[target])
    
    # train-trainの予測する（predict関数）
    shomatsu_train_pred_y = tmp_shomatsu_lgr_clf.predict(shomatsu_lgr_train_train_df[shomatsu_lgr_features])
    
    # train-trainの予測する（predict_proba関数）
    shomatsu_train_proba_y = tmp_shomatsu_lgr_clf.predict_proba(shomatsu_lgr_train_train_df[shomatsu_lgr_features]).T[1]
    
    # train-validの予測する（predict関数）
    shomatsu_valid_pred_y = tmp_shomatsu_lgr_clf.predict(shomatsu_lgr_train_valid_df[shomatsu_lgr_features])
    
    # train-validの予測する（predict_proba関数）
    shomatsu_valid_proba_y = tmp_shomatsu_lgr_clf.predict_proba(shomatsu_lgr_train_valid_df[shomatsu_lgr_features]).T[1]
    
    # train-trainの検証する
    shomatsu_train_accuracy_val = accuracy_score(shomatsu_lgr_train_train_df[target], shomatsu_train_pred_y)
    shomatsu_train_auc_val = roc_auc_score(shomatsu_lgr_train_train_df[target], shomatsu_train_proba_y)
    
    # train-validの検証する
    shomatsu_valid_accuracy_val = accuracy_score(shomatsu_lgr_train_valid_df[target], shomatsu_valid_pred_y)
    shomatsu_valid_auc_val = roc_auc_score(shomatsu_lgr_train_valid_df[target], shomatsu_valid_proba_y)
    
    # リストに格納する
    shomatsu_gs_result_list += [[
        params_dict,
        params_dict["C"],
        shomatsu_train_accuracy_val,
        shomatsu_train_auc_val,
        shomatsu_valid_accuracy_val,
        shomatsu_valid_auc_val
    ]]

In [ ]:
# 結果をDataFrameに格納する．
shomatsu_gs_result_df = pd.DataFrame(
    shomatsu_gs_result_list,
    columns=["params_dict", "L2_regularization_strength", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"]
)

# 各パラメータ値ごとのvalidのAUCを確認する.
shomatsu_gs_result_df.sort_values(by="valid_auc", ascending=False).reset_index(drop=True).head(10)

In [ ]:
# Accuracyの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
plt.plot(shomatsu_gs_result_df["L2_regularization_strength"], shomatsu_gs_result_df["train_accuracy"], label="train_accuracy") # train accuracy
plt.plot(shomatsu_gs_result_df["L2_regularization_strength"], shomatsu_gs_result_df["valid_accuracy"], label="valid_accuracy") # valid accuracy
plt.title("Accuracy\nlgr L2_regularization_strength tuning", fontsize=10)
plt.xlabel("L2_regularization_strength", fontsize=10)
plt.ylabel("Accuracy", fontsize=10)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# AUCの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
plt.plot(shomatsu_gs_result_df["L2_regularization_strength"], shomatsu_gs_result_df["train_auc"], label="train_auc") # train auc
plt.plot(shomatsu_gs_result_df["L2_regularization_strength"], shomatsu_gs_result_df["valid_auc"], label="valid_auc") # valid auc
plt.title("AUC\nlgr L2_regularization_strength tuning", fontsize=10)
plt.xlabel("L2_regularization_strength", fontsize=10)
plt.ylabel("AUC", fontsize=10)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# train-validのAUCが最も高いハイパーパラメータの組み合わせを変数に格納する.
shomatsu_lgr_best_idx = np.argmax(shomatsu_gs_result_df["valid_auc"])
shomatsu_lgr_best_params = shomatsu_gs_result_df["params_dict"].values[shomatsu_lgr_best_idx]
shomatsu_lgr_best_score = shomatsu_gs_result_df["valid_auc"].values[shomatsu_lgr_best_idx]

# 内容を確認
print("lgr_best_params:", shomatsu_lgr_best_params)
print("lgr_best_score:", shomatsu_lgr_best_score)

In [ ]:
# 最適化したハイパーパラメータを使用してモデルを定義する
shomatsu_lgr_clf = logr(**shomatsu_lgr_best_params, max_iter=1000)

# 学習する
shomatsu_lgr_clf.fit(shomatsu_lgr_train_df[shomatsu_lgr_features], shomatsu_lgr_train_df[target])

In [ ]:
# 標準化されていないことを確認する
shomatsu_lgr_train_df[shomatsu_lgr_features].describe()

In [ ]:
# 説明変数の標準化を行う
scaler = StandardScaler()
shomatsu_lgr_train_df[shomatsu_lgr_features] = scaler.fit_transform(shomatsu_lgr_train_df[shomatsu_lgr_features])

# 標準化されたことを確認する
shomatsu_lgr_train_df[shomatsu_lgr_features].describe()

In [ ]:
# 最適化したハイパーパラメータを使用してモデルを定義する
shomatsu_lgr_clf = logr(**shomatsu_lgr_best_params, max_iter=1000)

# 学習する
shomatsu_lgr_clf.fit(shomatsu_lgr_train_df[shomatsu_lgr_features], shomatsu_lgr_train_df[target])

In [ ]:
# 偏回帰係数を確認する.
shomatsu_lgr_coef_df = pd.DataFrame({"feature": shomatsu_lgr_features, "coefficient": shomatsu_lgr_clf.coef_[0]})
shomatsu_lgr_coef_df = shomatsu_lgr_coef_df.append(pd.DataFrame({"feature": "constant", "coefficient": shomatsu_lgr_clf.intercept_}))
shomatsu_lgr_coef_df["abs_coefficient"] = shomatsu_lgr_coef_df["coefficient"].abs()
shomatsu_lgr_coef_df = shomatsu_lgr_coef_df.sort_values(by="abs_coefficient", ascending=False).reset_index(drop=True)
shomatsu_lgr_coef_df.head(15)

In [ ]:
# trainの予測する
shomatsu_train_pred_y = shomatsu_lgr_clf.predict(shomatsu_lgr_train_df[shomatsu_lgr_features])
shomatsu_train_proba_y = shomatsu_lgr_clf.predict_proba(shomatsu_lgr_train_df[shomatsu_lgr_features]).T[1]

# trainの検証する
shomatsu_train_accuracy_val = accuracy_score(shomatsu_lgr_train_df[target], shomatsu_train_pred_y)
shomatsu_train_auc_val = roc_auc_score(shomatsu_lgr_train_df[target], shomatsu_train_proba_y)
shomatsu_train_accuracy_val, shomatsu_train_auc_val

In [ ]:
# testの予測する
shomatsu_test_pred_y = shomatsu_lgr_clf.predict(shomatsu_lgr_test_df[shomatsu_lgr_features])
shomatsu_test_proba_y = shomatsu_lgr_clf.predict_proba(shomatsu_lgr_test_df[shomatsu_lgr_features]).T[1]

In [ ]:
# sample submitデータを読み込む
gi_sample_submit_df = pd.read_csv("../input/gi_sample_submit.csv")

In [ ]:
# submit向けDataFrameを作成し、列に予測確率を格納する
shomatsu_submit_df = shomatsu_lgr_test_df.copy()[["customer_id"]]
shomatsu_submit_df["buy_proba"] = shomatsu_test_proba_y
shomatsu_submit_df.head()

In [ ]:
gi_sample_submit_df.shape, shomatsu_submit_df.shape

In [ ]:
# outputディレクトリにsubmit用ファイルを出力する
shomatsu_submit_df.to_csv(f"../output/shomatsu_submit_lgr.csv", encoding="utf-8", index=False)

### 【解説】

- ロジスティック回帰も決定木と同様で、機械学習モデル構築のプロセスは書籍本編とほぼ同じプログラムとなります。
- 従って、元となるDataFrame（特徴量）を入れ替えてあげれば、そのまま同様のプログラムを活用することが可能です。